In [1]:
## Set notebook to auto reload updated modules
%load_ext autoreload
%autoreload 2

In [2]:
%env ENV_FOR_DYNACONF=dev

env: ENV_FOR_DYNACONF=dev


In [19]:
from owm_bot.core.config import (
    AppSettings,
    settings,
    OpenweathermapSettings,
    owm_settings,
)

In [4]:
LOGGING_LEVEL = settings.log_level

In [5]:
import typing
from pathlib import Path
import json
import random

import logging
import logging.config

In [6]:
import red_logging

_formatter = red_logging.get_formatter_config(
    name="nb",
    fmt="%(asctime)s | %(levelname)s | %(message)s",
    datefmt=red_logging.fmts.DATE_FMT_DATE_ONLY,
)
console_handler = red_logging.get_streamhandler_config(
    name="console", formatter="nb", level="WARNING"
)
_handler = red_logging.get_streamhandler_config(
    name="console_nb", formatter="nb", level="DEBUG"
)
_logger = red_logging.get_logger_config(
    name="nb", handlers=["console_nb"], level=LOGGING_LEVEL
)

In [7]:
logging_config = red_logging.assemble_configdict(
    disable_existing_loggers=True,
    formatters=[_formatter],
    handlers=[_handler, console_handler],
    loggers=[_logger],
)

In [8]:
logging.config.dictConfig(logging_config)
log = logging.getLogger("nb")

In [13]:
from owm_bot.core import (
    DATA_DIR,
    CACHE_DIR,
    SERIALIZE_DIR,
    PQ_DIR,
    ENSURE_DIRS,
    OUTPUT_DIR,
)
from owm_bot.core import (
    PQ_ENGINE,
    OPENWEATHERMAP_BASE_URL,
    OPENWEATHERMAP_ONECALL_URL,
    OPENWEATHERMAP_GEO_URL,
)
from owm_bot.setup import setup_dirs
from owm_bot.domain.Location import (
    JsonLocation,
    JsonLocationsLoader,
    OwmGeoLookup,
)

from owm_bot.domain.Weather.current import OWMCurrentWeather
from owm_bot.domain.Weather.forecast import (
    OWMForecastWeather,
    OWMForecastWeatherEntry,
)
from owm_bot.utils import data_utils
from owm_bot.controllers import OpenWeathermapController
from owm_bot.weather import (
    CurrentWeatherPQFileController,
    ForecastWeatherPQFileController,
)

In [12]:
import pandas as pd
from red_utils.ext.dataframe_utils import pandas_utils

pd.set_option("display.max_columns", None)

In [11]:
setup_dirs(ensure_dirs=ENSURE_DIRS)

In [18]:
GET_WEATHER: bool = False

---

In [25]:
if GET_WEATHER:
    with OpenWeathermapController(units=owm_settings.units) as owm_ctl:
        current_weather: OWMCurrentWeather = owm_ctl.current_weather(save_pq=True)
        # log.debug(f"Current weather: {current_weather}")
        log.info("Refreshed current weather")

        weather_forecast: OWMForecastWeather = owm_ctl.weather_forecast(save_pq=True)
        # log.debug(f"Weather forecast: {weather_forecast}")
        log.info("Refreshed weather forecast")

2024-06-09 | INFO | Refreshed current weather
2024-06-09 | INFO | Refreshed weather forecast


In [26]:
with CurrentWeatherPQFileController() as currentweather_ctl:
    current_weather_df = currentweather_ctl.df

In [27]:
current_weather_df.head(5)

,coord,weather,base,main,visibility,wind,rain,clouds,sys,timezone,id,name,cod,timestamp
0,"{'lat': 41.3826, 'lon': -81.7964}","[{'description': 'overcast clouds', 'icon': '0...",stations,"{'feels_like': 65.14, 'humidity': 61, 'pressur...",10000,"{'deg': 20, 'gust': None, 'speed': 11.5}",None,{'all': 100},"{'country': 'US', 'id': 2007582, 'sunrise': 17...",-14400,5148326,Brook Park,200,2024-06-09 22:06:36
0,"{'lat': 41.3826, 'lon': -81.7964}","[{'description': 'overcast clouds', 'icon': '0...",stations,"{'feels_like': 65.14, 'humidity': 61, 'pressur...",10000,"{'deg': 20, 'gust': None, 'speed': 11.5}",None,{'all': 100},"{'country': 'US', 'id': 2007582, 'sunrise': 17...",-14400,5148326,Brook Park,200,2024-06-09 22:06:51


In [28]:
with ForecastWeatherPQFileController() as weatherforecast_ctl:
    weather_forecast_df = weatherforecast_ctl.df

In [29]:
weather_forecast_df.head(5)

,cod,message,cnt,entries
0,200,0,16,"[{'clouds': {'all': 100}, 'dt': 1717988400, 'd..."
0,200,0,16,"[{'clouds': {'all': 100}, 'dt': 1717988400, 'd..."
